In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import *

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup
import csv; import requests
import time
import pandas as pd
import numpy as np
import os
import psutil

import warnings
warnings.filterwarnings('ignore')

import datetime
import time

In [8]:
class kfc_web_scraper:
    def __init__(self):
        self.pick_up_css = "#app > div.kfc_view_box > div > div.__start_ordering_container > div > ul > li:nth-child(1) > a"
        self.drop_down_css = "#delivery > div > form > div > div > span > span > i"
        self.store_select_css = "ul.el-scrollbar__view.el-select-dropdown__list"
        self.drop_list_css = ".el-scrollbar__view"
        self.address_confirm_css = "html body.overflow div#app.__ishome div.kfc_view_box div.kfc_dialog div.__start_ordering_container div.__ordering_content div.tab-content div#delivery.tab-pane.active div.__pick_delivery_tab button.kfc_btn.__brand.__address_confirm_button"
        self.search_button = "#delivery > div > button.kfc_btn.__brand.__address_confirm_button"
        self.img = ".__product_item_image > a > .__product_im_img > img"
        self.name = ".__product_item_desc > a > h2"
        self.price = ".__product_item_desc > .__product_desc_price"
        self.main_menu = ".__main_menu_content"
        self.category = ".__main_content_section"
        self.items = ".__main_content_block > .__product_item:not([style])"
        self.url = "https://www.kfcmongolia.com/index"
        self.store_list_dict = ''
        self.main_Data_frame = pd.DataFrame(columns=["names", "prices", "categories", "image", "store"])
        
    def get_browser(self):
        options = webdriver.ChromeOptions()
        options.add_argument('headless')
        options.add_argument('window-size=1000x1000')
        options.add_argument("disable-gpu")
        
        self.driver = webdriver.Chrome(
                                        os.getcwd() + '\chromedriver', 
                                        options=options,
#                                         service=Service(ChromeDriverManager().install())
        )
        self.driver.get(self.url)
        self.driver_process = psutil.Process(self.driver.service.process.pid)
        
        pick_up = self.driver.find_element(By.CSS_SELECTOR, self.pick_up_css)
        pick_up.click()
        
        drop_down = self.driver.find_element(By.CSS_SELECTOR, self.drop_down_css)
        drop_down.click()

    def get_store_dict(self):
        self.get_browser()
        
        time.sleep(2)
        stores = self.driver.find_element(By.CSS_SELECTOR, self.store_select_css).text
        store_list = stores.split('\n')
        self.store_list_dict = {i+1:s for i,s in enumerate(store_list)}
        
        self.quit_browser()

    def get_store_items(self):
        
        self.get_store_dict()
        stores = len(self.store_list_dict)
        initial_index = 0
        error = 0
        
        print('stores', stores)
        
        while (stores > 0) and (initial_index != stores) and (error != 10):
            try:
                for i in range(initial_index, stores):
                    item_imgs = []
                    item_names = []
                    item_prices = []
                    item_categories = []
                    item_store = []

                    self.get_browser()

                    WebDriverWait(self.driver, 100).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'li.el-select-dropdown__item')))
                    input_store_names = self.driver.find_elements(By.CSS_SELECTOR, 'li.el-select-dropdown__item')
                    input_store_names[i].click()
                    
                    print(len(input_store_names), i, input_store_names[i].text)

                    store_name = input_store_names[i].text
                    search_button = self.driver.find_element(By.CSS_SELECTOR, self.search_button)
                    search_button.click()

                    time.sleep(2)
                    get_main_menu = self.driver.find_element(By.CSS_SELECTOR, self.main_menu)
                    WebDriverWait(self.driver, 100).until(EC.presence_of_element_located((By.CSS_SELECTOR, self.category)))
                    get_categories = get_main_menu.find_elements(By.CSS_SELECTOR, self.category)

                    for j in range(len(get_categories)):

                        get_items = get_categories[j].find_elements(By.CSS_SELECTOR, self.items)
                        category = get_categories[j].find_element(By.CSS_SELECTOR, 'h3').text

                        for k in range(len(get_items)):
                            img = get_items[k].find_element(By.CSS_SELECTOR, self.img).get_attribute("src")
                            name = get_items[k].find_element(By.CSS_SELECTOR, self.name).text
                            price = get_items[k].find_element(By.CSS_SELECTOR, self.price).text

                            if name != '':
                                item_imgs.append(img)
                                item_categories.append(category)
                                item_names.append(name)
                                item_prices.append(price)
                                item_store.append(store_name)

                    self.quit_browser()

                    temp_DataFrame = pd.DataFrame({"names": item_names, 
                                                   "prices": item_prices,
                                                   "categories": item_categories,
                                                   "image": item_imgs, 
                                                   "store": item_store
                                                  })
                    self.main_Data_frame = pd.concat([self.main_Data_frame, temp_DataFrame])
                    initial_index = initial_index + 1
                    error = 0
            except:
                self.quit_browser()
                error = error + 1
                if error > 8:
                    initial_index = initial_index + 1
                    print("----- skipped to next -----")
                    error = 0
                print(f"initial_index = {initial_index}, error = {error}", )
                continue
        
        print("done...........")
        return self.main_Data_frame
            
    def quit_browser(self, ):
        self.driver.quit()

In [9]:
start = time.time()

test = kfc_web_scraper()
df = test.get_store_items()

end = time.time()
elapsed = end - start
print(f'Time taken: {elapsed:.6f} seconds')

stores 16
16 0 KFC-БИЗНЕСС ПЛАЗА /Төв номын сан/
16 1 KFC-Смарт /Өргөө кино театрын эсрэг талд/
initial_index = 1, error = 1
16 1 KFC-Смарт /Өргөө кино театрын эсрэг талд/
16 2 KFC-Сөүл (Хүүхдийн 100)
16 3 KFC-Тэнгис /Тэнгис кино театрын хажууд/
initial_index = 3, error = 1
16 3 KFC-Тэнгис /Тэнгис кино театрын хажууд/
16 4 KFC-Намянжу/(ХӨСҮТ дэргэд)
16 5 KFC-Энхтайван (24 цагийн салбар)
initial_index = 5, error = 1
16 5 KFC-Энхтайван (24 цагийн салбар)
16 6 KFC-Хархорин /Хархорин захын буудал/
initial_index = 7, error = 1
16 7 KFC-Зайсан салбар
initial_index = 7, error = 2
16 7 KFC-Зайсан салбар
initial_index = 8, error = 1
16 8 KFC-Сансар /24 Цагийн салбар/
initial_index = 8, error = 2
initial_index = 8, error = 3
16 8 KFC-Сансар /24 Цагийн салбар/
initial_index = 9, error = 1
16 9 KFC-Морьтон салбар /19-н Буудал/
initial_index = 9, error = 2
16 9 KFC-Морьтон салбар /19-н Буудал/
initial_index = 9, error = 3
initial_index = 9, error = 4
16 9 KFC-Морьтон салбар /19-н Буудал/
initial_in

In [10]:
df['store'].value_counts()

KFC-Яармаг /Яармаг Номингийн 1 давхарт/        55
KFC-Морьтон салбар /19-н Буудал/               54
KFC-21 Салбар                                  53
KFC-Баянхошуу салбар                           53
KFC-БИЗНЕСС ПЛАЗА /Төв номын сан/              52
KFC-Зайсан салбар                              52
KFC-Хоум Плаза /Хан-Уул/ - 24 цагийн салбар    51
KFC-Смарт /Өргөө кино театрын эсрэг талд/      50
KFC-ГАЛЛЕРИА /Галлериа моол 1 давхарт/         50
KFC-Энхтайван (24 цагийн салбар)               49
KFC-Намянжу/(ХӨСҮТ дэргэд)                     48
KFC-Хархорин /Хархорин захын буудал/           48
KFC-Сансар /24 Цагийн салбар/                  48
KFC-Сөүл (Хүүхдийн 100)                        47
KFC-Тэнгис /Тэнгис кино театрын хажууд/        43
Name: store, dtype: int64

In [11]:
len(df['store'].unique())

15

In [12]:
df.to_excel(f'kfc_mongolia_price_data_{datetime.datetime.now().year}{datetime.datetime.now().month}{datetime.datetime.now().day}.xlsx')

,names,prices,categories,image,store
0,10ш НАГГЕТ,"7,500₮",ДАН БVТЭЭГДЭХVVН,https://imgur.com/upY59gK.png,KFC-БИЗНЕСС ПЛАЗА /Төв номын сан/
1,20ш НАГГЕТ,"14,000₮",ДАН БVТЭЭГДЭХVVН,https://imgur.com/mMZXiqy.png,KFC-БИЗНЕСС ПЛАЗА /Төв номын сан/
2,ТВИСТЕР,"7,000₮",ДАН БVТЭЭГДЭХVVН,https://imgur.com/YVsecYB.png,KFC-БИЗНЕСС ПЛАЗА /Төв номын сан/
3,ЗИНГЕР БУРГЕР,"7,000₮",ДАН БVТЭЭГДЭХVVН,https://imgur.com/zOXCxeX.png,KFC-БИЗНЕСС ПЛАЗА /Төв номын сан/
4,ФИЛЛЕ БУРГЕР,"7,000₮",ДАН БVТЭЭГДЭХVVН,https://imgur.com/cWqe3m3.png,KFC-БИЗНЕСС ПЛАЗА /Төв номын сан/
...,...,...,...,...,...
48,MOUNTAIN DEW R,"1,000₮",УНДАА,https://imgur.com/bknkfQA.png,KFC-Баянхошуу салбар
49,ЦАЙ,"1,500₮",УНДАА,https://imgur.com/33LiW7o.png,KFC-Баянхошуу салбар
50,ЦЭВЭР УС,"1,000₮",УНДАА,https://imgur.com/1iiYLdP.png,KFC-Баянхошуу салбар
51,PEPSI L,"1,500₮",УНДАА,https://imgur.com/G7foBq9.png,KFC-Баянхошуу салбар


In [16]:
os.listdir()

['.git',
 '.ipynb_checkpoints',
 '20230404 KFC price research_test.xlsx',
 'chromedriver.exe',
 'chromedriver_win32',
 'chromedriver_win32.zip',
 'geckodriver.log',
 'kfc_mongolia_price_data.xlsx',
 'kfc_mongolia_price_data_2023418.xlsx',
 'kfc_mongolia_price_data_2023421.xlsx',
 'LICENSE.chromedriver',
 'new_kfc_scrap.ipynb',
 'old_kfc_scrap.ipynb']

In [138]:
df_230404 = pd.read_excel('20230404 KFC price research_test.xlsx')
df_230404 = df_230404.drop(['KFC - Menu Prices', 'Unnamed: 1', 'Unnamed: 3',
                               'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8',
                               'Unnamed: 9'], axis=1)
df_230404.columns.values[-1] = 'Price'
df_230404.columns.values[-2] = 'Name'
df_230404 = df_230404.drop([0, 1])
df_230404.head()

,Name,Price
2,Tender burger,3500
3,Hash brown & egg burger,4000
4,Egg twister,8000
5,Egg burger,5500
6,Fillet burger,7000


In [141]:
df['names'] = df['names'].str.lower()
df[df['names'].str.contains("тэндэр")].head()

,names,prices,categories,image,store
12,тэндэр сет,"8,500₮",СЕТ БVТЭЭГДЭХVVН,https://imgur.com/Z9fwkIq.png,KFC-БИЗНЕСС ПЛАЗА /Төв номын сан/
14,хүүхдийн тэндэртэй сэт,"11,000₮",СЕТ БVТЭЭГДЭХVVН,https://imgur.com/ebBR49c.png,KFC-БИЗНЕСС ПЛАЗА /Төв номын сан/
17,өег тэндэртэй багц,"16,500₮",ӨЕГ БАГЦУУД,https://imgur.com/zuWBE9U.png,KFC-БИЗНЕСС ПЛАЗА /Төв номын сан/
22,тэндэр арвин багц,"50,000₮",АРВИН БАГЦУУД,https://imgur.com/1ilRLTh.png,KFC-БИЗНЕСС ПЛАЗА /Төв номын сан/
12,тэндэр сет,"8,500₮",СЕТ БVТЭЭГДЭХVVН,https://imgur.com/Z9fwkIq.png,KFC-Смарт /Өргөө кино театрын эсрэг талд/
